In [1]:
import tensorflow as tf
import os
from os.path import isfile, join
import numpy as np
import shutil
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
import subprocess

Copy dataset to arrange audio and noise in different folders

In [2]:
!cp -r "../input/speaker-recognition-dataset" ./

Get the data directories

In [3]:

data_directory = "./speaker-recognition-dataset/16000_pcm_speeches"
audio_folder = "audio"
noise_folder = "noise"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [4]:
audio_path

'./speaker-recognition-dataset/16000_pcm_speeches/audio'

set all the parameters for training and other purposes

In [5]:
valid_split = 0.1

shuffle_seed = 43

sample_rate = 16000

scale = 0.5

batch_size = 128

epochs = 15

arrange audio and noise

In [6]:
for folder in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, folder)):
        if folder in [audio_folder, noise_folder]:
            
            continue
        elif folder in ["other", "_background_noise_"]:
            
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(noise_path, folder),
            )
        else:
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(audio_path, folder),
            )


Get the list of all noise files

In [7]:

noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [8]:
noise_paths

['speaker-recognition-dataset/16000_pcm_speeches/noise/other/pink_noise.wav',
 'speaker-recognition-dataset/16000_pcm_speeches/noise/other/exercise_bike.wav',
 'speaker-recognition-dataset/16000_pcm_speeches/noise/_background_noise_/doing_the_dishes.wav',
 'speaker-recognition-dataset/16000_pcm_speeches/noise/_background_noise_/running_tap.wav',
 'speaker-recognition-dataset/16000_pcm_speeches/noise/_background_noise_/dude_miaowing.wav',
 'speaker-recognition-dataset/16000_pcm_speeches/noise/_background_noise_/10convert.com_Audience-Claps_daSG5fwdA7o.wav']

**Split noise into chunks of 16,000 steps each**

In [9]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

In [10]:
os.system(command)
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)
noises

<tf.Tensor: shape=(354, 16000, 1), dtype=float32, numpy=
array([[[ 0.6796875 ],
        [ 0.32202148],
        [ 0.41156006],
        ...,
        [-0.07870483],
        [-0.00308228],
        [ 0.1557312 ]],

       [[ 0.12347412],
        [-0.0255127 ],
        [-0.06317139],
        ...,
        [ 0.03457642],
        [ 0.2083435 ],
        [ 0.4513855 ]],

       [[ 0.42971802],
        [ 0.2899475 ],
        [ 0.40582275],
        ...,
        [-0.59317017],
        [-0.49490356],
        [-0.60446167]],

       ...,

       [[ 0.04284668],
        [ 0.00845337],
        [-0.00296021],
        ...,
        [ 0.03219604],
        [-0.00515747],
        [-0.03042603]],

       [[-0.04034424],
        [-0.04818726],
        [-0.05633545],
        ...,
        [ 0.00631714],
        [ 0.03829956],
        [ 0.02713013]],

       [[-0.02148438],
        [-0.05950928],
        [-0.05187988],
        ...,
        [-0.00717163],
        [-0.01553345],
        [-0.01739502]]], dtype=float3

**DATASET GENERATION**

In [11]:
def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [12]:
def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, sample_rate)
    return audio

**add noise to dataset**

In [13]:
def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        audio = audio + noise * prop * scale
        print(audio)
    return audio

In [14]:
def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

In [15]:

class_names = os.listdir(audio_path)
print(class_names,)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Speaker:",(name))
    dir_path = Path(audio_path) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

['Benjamin_Netanyau', 'Julia_Gillard', 'Magaret_Tarcher', 'Jens_Stoltenberg', 'Nelson_Mandela']
Speaker: Benjamin_Netanyau
Speaker: Julia_Gillard
Speaker: Magaret_Tarcher
Speaker: Jens_Stoltenberg
Speaker: Nelson_Mandela


In [16]:
# Shuffle to generate random data
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

In [17]:
# Split into training and validation
num_val_samples = int(valid_split * len(audio_paths))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]


valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

In [18]:
# Create datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

**feature Extraction**

In [19]:
# Add noise to the training set
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=scale), y),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.experimental.AUTOTUNE)

Tensor("add:0", shape=(None, 16000, 1), dtype=float32)


**Model**

In [20]:
from tensorflow.keras.layers import Conv1D

In [21]:
def residual_block(x, filters, conv_num = 3, activation = "relu"):
    s = keras.layers.Conv1D(filters, 1, padding = "same")(x)
    
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
        x = keras.layers.Activation(activation)(x)
    
    x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    
    return keras.layers.MaxPool1D(pool_size = 2, strides = 2)(x)

def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape = input_shape, name = "input")
    
    x = residual_block(inputs, 16, 2)
    x = residual_block(inputs, 32, 2)
    x = residual_block(inputs, 64, 3)
    x = residual_block(inputs, 128, 3)
    x = residual_block(inputs, 128, 3)
    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    
    outputs = keras.layers.Dense(num_classes, activation = "softmax", name = "output")(x)
    
    return keras.models.Model(inputs = inputs, outputs = outputs)

model = build_model((sample_rate // 2, 1), len(class_names))

model.summary()

model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(model_save_filename, monitor="val_accuracy", save_best_only=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 8000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_15 (Conv1D)              (None, 8000, 128)    512         input[0][0]                      
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 8000, 128)    0           conv1d_15[0][0]                  
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 8000, 128)    49280       activation_10[0][0]              
______________________________________________________________________________________________

**Training**

In [22]:
# epochs=1

In [23]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/15
53/53 [==============================] - 73s 1s/step - loss: 18.1698 - accuracy: 0.6147 - val_loss: 0.3662 - val_accuracy: 0.8573


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/15
53/53 [==============================] - 67s 1s/step - loss: 0.2369 - accuracy: 0.9119 - val_loss: 0.1356 - val_accuracy: 0.9533
Epoch 3/15
53/53 [==============================] - 69s 1s/step - loss: 0.1739 - accuracy: 0.9317 - val_loss: 0.1051 - val_accuracy: 0.9653
Epoch 4/15
53/53 [==============================] - 67s 1s/step - loss: 0.1222 - accuracy: 0.9541 - val_loss: 0.1536 - val_accuracy: 0.9413
Epoch 5/15
53/53 [==============================] - 66s 1s/step - loss: 0.1409 - accuracy: 0.9507 - val_loss: 0.1200 - val_accuracy: 0.9667
Epoch 6/15
53/53 [==============================] - 67s 1s/step - loss: 0.0820 - accuracy: 0.9714 - val_loss: 0.0552 - val_accuracy: 0.9827
Epoch 7/15
53/53 [==============================] - 67s 1s/step - loss: 0.0695 - accuracy: 0.9756 - val_loss: 0.0570 - val_accuracy: 0.9867
Epoch 8/15
53/53 [==============================] - 67s 1s/step - loss: 0.0717 - accuracy: 0.9732 - val_loss: 0.0411 - val_accuracy: 0.9867
Epoch 9/15
53/53 [==

**Accuracy**

In [24]:
print("Accuracy of model:",model.evaluate(valid_ds))

24/24 [==============================] - 6s 249ms/step - loss: 0.0431 - accuracy: 0.9880
Accuracy of model: [0.04310839995741844, 0.9879999756813049]


**Predict**

In [25]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

for audios, labels in test_ds.take(1):
    ffts = audio_to_fft(audios)
    y_pred = model.predict(ffts)
    rnd = np.random.randint(0, batch_size, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        if labels[index] ==y_pred[index]:
            print("Welcome")
        else:
            print("Sorry")
        print("The speaker is" if labels[index] == y_pred[index] else "", class_names[y_pred[index]])

Tensor("add:0", shape=(None, 16000, 1), dtype=float32)
Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau
Welcome
The speaker is Benjamin_Netanyau
Speaker: Julia_Gillard	Predicted: Julia_Gillard
Welcome
The speaker is Julia_Gillard
Speaker: Julia_Gillard	Predicted: Julia_Gillard
Welcome
The speaker is Julia_Gillard
Speaker: Julia_Gillard	Predicted: Julia_Gillard
Welcome
The speaker is Julia_Gillard
Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau
Welcome
The speaker is Benjamin_Netanyau
Speaker: Julia_Gillard	Predicted: Julia_Gillard
Welcome
The speaker is Julia_Gillard
Speaker: Nelson_Mandela	Predicted: Nelson_Mandela
Welcome
The speaker is Nelson_Mandela
Speaker: Magaret_Tarcher	Predicted: Magaret_Tarcher
Welcome
The speaker is Magaret_Tarcher
Speaker: Nelson_Mandela	Predicted: Nelson_Mandela
Welcome
The speaker is Nelson_Mandela
Speaker: Benjamin_Netanyau	Predicted: Benjamin_Netanyau
Welcome
The speaker is Benjamin_Netanyau


**Predcit the speaker from the test dataset for real time pred.**

In [26]:
 def paths_to_dataset(audio_paths):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    return tf.data.Dataset.zip((path_ds))

def predict(path, labels):
    test = paths_and_labels_to_dataset(path, labels)


    test = test.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
    )
    test = test.prefetch(tf.data.experimental.AUTOTUNE)


    test = test.map(lambda x, y: (add_noise(x, noises, scale=scale), y))

    for audios, labels in test.take(1):
        ffts = audio_to_fft(audios)
        y_pred = model.predict(ffts)
        rnd = np.random.randint(0, 1, 1)
        audios = audios.numpy()[rnd, :]
        labels = labels.numpy()[rnd]
        y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(1):
            print(
            "Speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
            "[92m",y_pred[index],
                "[92m", y_pred[index]
                )
            )
            
            print("Speaker Predicted:",class_names[y_pred[index]])
    return class_names[y_pred[index]]



In [27]:
import csv
from datetime import datetime

In [28]:
# Sample attendance data
attendance_data = [
#     {"Name": "", "Status": ""}
#     {"Name": "Jane", "Time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")},
#     {"Name": "Mike", "Time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")},
#     {"Name": "Emily", "Time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")},
#     {"Name": "Alex", "Time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
]

# Define the output file path
output_file = "attendance.csv"

# Define the field names for the CSV file
field_names = ["Name", "Status"]

# Write attendance data to CSV file
with open(output_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=field_names)

    # Write the header
    writer.writeheader()

    # Write the attendance data
    writer.writerows(attendance_data)

print("Attendance file created successfully.")

Attendance file created successfully.


In [29]:
path = ["/kaggle/input/speaker-recognition-dataset/16000_pcm_speeches/Magaret_Tarcher/1000.wav"]
labels = ["unknown"]
predicted_speaker = ""
try:
    predicted_speaker = predict(path, labels)
    print(predicted_speaker)
except:
    print("Error! Check if the file correctly passed or not!")

Tensor("add:0", shape=(None, 16000, 1), dtype=float32)
Speaker: 2	Predicted: 2
Speaker Predicted: Magaret_Tarcher
Magaret_Tarcher


In [30]:
print(predicted_speaker)

Magaret_Tarcher


In [31]:
def write_to_csv(file_path, predicted_speaker):
    # Get the current timestamp
#     current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    present="present"

    # Create a row with the predicted speaker and their status
    row = [predicted_speaker,present]

    # Write the row to the CSV file
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

In [32]:
# Path to the CSV file
csv_file = '/kaggle/working/attendance.csv'

In [33]:
# Write the predicted speaker and timestamp to the CSV file
write_to_csv(csv_file, predicted_speaker)
print("Entry added to the CSV file.")

Entry added to the CSV file.
